In [58]:
from __future__ import division
import os
import numpy as np
import pandas as pd
from skimage.io import imread, imshow, imsave, show
from IPython.display import Image, HTML
import operator
pd.set_option('display.max_colwidth', -1)

In [59]:
from constants import LABELS, WEATHER_IDX

## Load data

In [60]:
expected_tags = {}
all_tags = set(LABELS)
weather_tags = set([t for i, t in enumerate(LABELS) if i in WEATHER_IDX])
all_tags = weather_tags

with open("./rawInput/train.csv", "r") as train_labels:
    train_labels.readline()
    for line in train_labels:
        img, tags = line.rstrip("\n").split(",")
        #expected_tags[img] = tags.split(" ")
        expected_tags[img] = [t for t in tags.split(" ") if t in weather_tags]
print(len(expected_tags))

40479


In [61]:
predicted_tags = {}
with open("./backups/ekami/train-predict.csv", "r") as predict_labels:
    predict_labels.readline()
    for line in predict_labels:
        img, tags = line.rstrip("\n").split(",")
        #predicted_tags[img] = tags.split(" ")
        #predicted_tags[img] = [t for t in tags.split(" ") if t not in weather_tags] + [t for t in expected_tags[img] if t in weather_tags]
        predicted_tags[img] = [t for t in tags.split(" ") if t in weather_tags]
print(len(predicted_tags))

40479


In [62]:
predicted_proba = {}
with open("./backups/ekami/train-predict-raw.csv", "r") as predict_proba:
    tags = predict_proba.readline().rstrip("\n").split(",")[1].split(" ")
    for line in predict_proba:
        img, proba = line.rstrip("\n").split(",")
        proba = proba.split(" ")
        predicted_proba[img] = { t: float(v) for t, v in zip(tags, proba) }
        #predicted_tags[img] = [tag for tag in predicted_proba[img] if predicted_proba[img][tag] > 0.2 and tag in weather_tags]
        #predicted_tags[img] = [tag for tag in predicted_proba[img] if predicted_proba[img][tag] > 0.2 and tag not in weather_tags] + [t for t in expected_tags[img] if t in weather_tags]
print(len(predicted_proba))
# recompute from proba


40479


In [63]:
training_set = set([])
validation_set = set([])
with open("./backups/ekami/training-files.csv", "r") as training_files:
    original_training_set = set(training_files.readline().strip().split(","))
    for f in os.listdir("./rawInput/train-jpg"):
        filename = f.split(".")[0]
        s = None
        if filename in original_training_set:
            s = training_set
        elif filename in predicted_tags:
            s = validation_set
        if s is not None:
            s.add(filename)
#         for i in range(8):
#             s.add("{name}--{i}".format(name=filename, i=i))

print("total images:", len(os.listdir("./rawInput/train-jpg")))
print("training set:", len(training_set))
print("validation set:", len(validation_set))

total images: 40479
training set: 36431
validation set: 4048


## Read predictions

### Per label score

In [64]:
# First value is number of correctly predicted
# Second is number of not detected (false negative)
# Third is number of inaccurately predicted (false positive)
tags_score = {t: { "tp":0, "fn": 0, "fp": 0} for t in all_tags}

def scores(tp, fp, fn):
    p = tp / max(1, (tp + fp))
    r = tp / max(1, (tp + fn))
    b = 2
    if p == 0 or r == 0:
        f2 = 0
    else:
        b = 2
        f2 = (1 + b**2) * p * r / (b**2*p + r)
    return p, r, f2
    
for img in validation_set:
    true_tags = expected_tags[img]
    pred_tags = predicted_tags[img]
    for t in true_tags:
        if t in pred_tags:
            tags_score[t]["tp"] += 1
        else:
            tags_score[t]["fn"] += 1
    for t in pred_tags:
        if t not in true_tags:
            tags_score[t]["fp"] += 1

data = []
for tag in all_tags:
    s = tags_score[tag]
    p, r, f2 = scores(s["tp"], s["fp"], s["fn"])
    data.append([tag, s["tp"], s["fp"], s["fn"], p, r, f2])
data.sort(key = lambda l:l[-1])
data = np.array(data)
columns = ["tp", "fp", "fn", "validation precision", "validation recall", "f2"]
df = pd.DataFrame(data=data[:, 1:], index=data[:, 0], columns=columns)
df

,tp,fp,fn,validation precision,validation recall,f2
haze,229,157,44,0.5932642487046632,0.8388278388278388,0.7746955345060892
cloudy,183,101,7,0.6443661971830986,0.9631578947368421,0.8764367816091954
partly_cloudy,693,154,21,0.8181818181818182,0.9705882352941176,0.9357277882797732
clear,2842,204,29,0.933026920551543,0.98989898989899,0.9779766001376462


## F2 score

In [65]:
import numpy as np
from sklearn.metrics import fbeta_score
def F2Score(predicted, actual):
    # see https://www.kaggle.com/c/planet-understanding-the-amazon-from-space#evaluation
    predicted = set(predicted)
    actual = set(actual)
    tp = len(predicted & actual)
    tn = len(LABELS) - len(predicted | actual)
    fp = len(predicted) - tp
    fn = (len(LABELS) - len(predicted)) - tn
    return scores(tp, fp, fn)[2]

def scoreSet(data_set):
    print("scoreSet", len(data_set), len(expected_tags), len(predicted_tags))
    scores = []
    f2, n = 0, 0
    for img in data_set:
        real_tags = expected_tags[img]
        pred_tags = predicted_tags[img]
        s = F2Score(pred_tags, real_tags)
        f2 += s
        scores.append({ "img": img, "f2": s })
        n += 1
    return f2 / n, scores

In [66]:
training_score, training_scores = scoreSet(training_set)
validation_score, validation_scores = scoreSet(validation_set)
training_scores.sort(key = lambda l: l["f2"])
validation_scores.sort(key = lambda l: l["f2"])

print("F2 score for training set", training_score)
print("F2 score validation set", validation_score)

scoreSet 36431 40479 40479
scoreSet 4048 40479 40479
F2 score for training set 0.955127828079401
F2 score validation set 0.9544572275550608


## Prediction per image

In [67]:
short_labels = {"agriculture": "ag","artisinal_mine": "a_mi","bare_ground": "b_gr","blooming": "bloom","blow_down": "blow","clear": "clea","cloudy": "clou","conventional_mine": "c_mi","cultivation": "cult","habitation": "hab","haze": "haze","partly_cloudy": "p_clo","primary": "prim","road": "road","selective_logging": "s_log","slash_burn": "sl_b","water": "wat"}
short_labels = { t: short_labels[t] for t in short_labels if t in all_tags }

# return an HTML display for one prediction entry, making it green, red or grey
def getLabelPred(tag, img):
    truth = tag in expected_tags[img]
    pred = tag in predicted_tags[img]
    prob = predicted_proba[img][tag]
    isGood = truth == pred
    if truth == pred:
        cl = "green" if truth else "grey"
    else:
        cl = "red true" if truth else "red false"
    return '<span title="{tag}" class="{cl}">{val}</span>'.format(tag=tag, val=round(prob, 2), cl=cl)
        
# Return a dataframe the predictions
def getPrediction(data):
    img = data["img"]
    f2 = data["f2"]
    image = '<img src="./rawInput/train-jpg/{img}.jpg" class="input-image"/>'.format(img=img)
    
    return [
        img, 
        image, 
        f2
    ] + [getLabelPred(t, img) for t in all_tags]

table = []
originals = set([])
for data in validation_scores[:1000]:
    img = data["img"]
    original = img.split("--")[0]
    if original not in originals:
        table.append(getPrediction(data))
        originals.add(original)

table = np.array(table)

columns = ["image", "f2"] + [short_labels[t] for t in all_tags]
df = pd.DataFrame(data=table[:, 1:], index=table[:, 0], columns=columns)
style = "<style>.input-image{width: 50px; max-width: 250px !important;} .input-image:hover{width: 250px;} .green{color: green;} .grey{color: #e0e0e0;} .red{color: black;} .true{border-bottom: 3px solid green;} .false{border-bottom: 3px solid red;}</style>"
HTML(style + df.to_html(escape=False))

,image,f2,haze,clea,clou,p_clo
train_31030,,0,0.88,0.03,0.02,0.01
train_26623,,0,0.06,0.22,0.0,0.71
train_33133,,0,0.01,0.23,0.0,0.83
train_40311,,0,0.22,0.56,0.05,0.03
train_37463,,0,0.03,0.95,0.0,0.01
train_30690,,0,0.01,0.95,0.0,0.02
train_15294,,0,0.12,0.84,0.01,0.01
train_26134,,0,0.04,0.01,0.94,0.0
train_25773,,0,0.06,0.01,0.52,0.48
train_20602,,0,0.06,0.0,0.93,0.0
